<a href="https://colab.research.google.com/github/zahid214/ANN-backpropagation-python/blob/main/GAN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!apt-get install unrar -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [5]:
import argparse
import os
import numpy as np
import csv
import lpips
import torch.nn as nn
import torch

from torch.autograd import Variable
from skimage.metrics import peak_signal_noise_ratio as psnr_metric, structural_similarity as ssim_metric
from pytorch_msssim import ms_ssim
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets

In [10]:
from google.colab import files
import os
# Upload zip file of your dataset
# uploaded = files.upload()  # select data-10.zip from your computer
# Unzip it
!unrar x Real-data.rar /content/



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from Real-data.rar

Creating    /content/Real-data                                        OK
Creating    /content/Real-data/Drinking                               OK
Extracting  /content/Real-data/Drinking/image_11_Drinking_2120_1296_305_337.jpg       2%  OK 
Extracting  /content/Real-data/Drinking/image_12_Drinking_2115_1097_300_305.jpg       4%  OK 
Extracting  /content/Real-data/Drinking/image_12_Drinking_60_291_185_203.jpg       6%  OK 
Extracting  /content/Real-data/Drinking/image_13_Drinking_1856_32_194_185.jpg       7%  OK 
Extracting  /content/Real-data/Drinking/image_13_Drinking_2106_1073_300_319.jpg       9%  OK 
Extracting  /content/Real-data/Drinking/image_2_Drinking_2184_1467_310_194.jpg      11%  OK 
Extracting  /content/Real-data/Drinking/image_6_Drinking_2273_1422_261_223.jpg      13%  OK 
Extracting  /content/Real-data/Dr

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=1000, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=64, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=3, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
parser.add_argument('--data', type=str, default="/content/Real-data")
# opt = parser.parse_args()
opt, _ = parser.parse_known_args()
print(opt)

Namespace(n_epochs=1000, batch_size=64, lr=0.0002, b1=0.5, b2=0.999, n_cpu=8, latent_dim=100, img_size=64, channels=3, sample_interval=400, data='/content/Real-data')


In [20]:
img_shape = (opt.channels, opt.img_size, opt.img_size)
cuda = True if torch.cuda.is_available() else False
print(cuda)
# Make folders
os.makedirs("images", exist_ok=True)
os.makedirs("logs", exist_ok=True)
# Prepare CSV file
csv_file = "logs/training_log.csv"
fieldnames = ['epoch', 'batch', 'd_loss', 'g_loss', 'd_score', 'g_score', 'psnr', 'ssim', 'msssim', 'lpips', 'mos']


False


In [13]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()        )
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [14]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),        )
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

In [21]:
# Loss function
adversarial_loss = torch.nn.BCELoss()
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()
lpips_fn = None
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    lpips_fn = lpips.LPIPS(net='alex').cuda()
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    adversarial_loss.to(device)
    generator.to(device)
    discriminator.to(device)
    lpips_fn = lpips.LPIPS(net='alex').to(device)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:03<00:00, 75.1MB/s]


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/alex.pth


In [16]:
# ===== Custom Dataset =====
transform = transforms.Compose([
    transforms.Resize((opt.img_size, opt.img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)])
dataset = datasets.ImageFolder(root=opt.data, transform=transform)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=True)

In [17]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
results = {'d_loss': [], 'g_loss': [], 'd_score': [], 'g_score': [], 'psnr': [], 'ssim': [], 'msssim': [], 'lpips': [], 'mos': []}


In [22]:
# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    epoch_psnr = 0.0
    epoch_ssim = 0.0
    epoch_msssim = 0.0
    epoch_lpips = 0.0
    num_batches = 0
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Scores
        d_score = discriminator(real_imgs).mean().item()
        g_score = discriminator(gen_imgs).mean().item()

        # ---------------------
        #  Metrics (PSNR, SSIM) per batch
        # ---------------------
        real_norm = (real_imgs - real_imgs.min()) / (real_imgs.max() - real_imgs.min() + 1e-8)
        gen_norm  = (gen_imgs  - gen_imgs.min())  / (gen_imgs.max()  - gen_imgs.min()  + 1e-8)
        real_np = real_imgs[0].detach().cpu().numpy().transpose(1, 2, 0)
        gen_np = gen_imgs[0].detach().cpu().numpy().transpose(1, 2, 0)
        real_np = (real_np - real_np.min()) / (real_np.max() - real_np.min() + 1e-8)
        gen_np = (gen_np - gen_np.min()) / (gen_np.max() - gen_np.min() + 1e-8)

        psnr_val = psnr_metric(real_np, gen_np, data_range=1.0)
        ssim_val = ssim_metric(real_np, gen_np, data_range=1.0, channel_axis=-1)
        # msssim_val = ms_ssim(gen_norm, real_norm, data_range=1.0, size_average=True).item()
        msssim_val = ms_ssim(gen_norm, real_norm, data_range=1.0, size_average=True, win_size=3).item()
        real_lp = real_norm * 2 - 1; gen_lp  = gen_norm  * 2 - 1
        lpips_val = lpips_fn(gen_lp, real_lp).mean().item()

        epoch_psnr += psnr_val
        epoch_ssim += ssim_val
        epoch_msssim += msssim_val
        epoch_lpips += lpips_val
        num_batches += 1

        print(
            f"[Epoch {epoch}/{opt.n_epochs}] [Batch {i}/{len(dataloader)}] "
            f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}] "
            f"[D score: {d_score:.4f}] [G score: {g_score:.4f}] "
            f"[PSNR: {psnr_val:.4f}] [SSIM: {ssim_val:.4f}]"
        )

    avg_psnr = epoch_psnr / num_batches
    avg_ssim = epoch_ssim / num_batches
    avg_msssim = epoch_msssim / num_batches
    avg_lpips = epoch_lpips / num_batches
    proxy_mos = (0.4 * avg_ssim + 0.3 * avg_msssim + 0.3 * (1 - avg_lpips))

    # ---------------------
    # Save CSV per batch
    # ---------------------
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writerow({
            'epoch': epoch,
            'batch': i,
            'd_loss': d_loss.item(),
            'g_loss': g_loss.item(),
            'd_score': d_score,
            'g_score': g_score,
            'psnr': avg_psnr,
            'ssim': avg_ssim,
            'msssim': avg_msssim,
            'lpips': avg_lpips,
            'mos': proxy_mos})

    # ---------------------
    # Save ONE image at the end of each epoch (from last batch)
    # ---------------------
    # save_path = f"images/epoch_{epoch}.png"
    # save_image(gen_imgs.data[0], save_path, normalize=True)
    # print(f"✅ Epoch {epoch} finished — image saved at {save_path}")


    # Save only one generated image per epoch (first sample in the batch)
    for im in range(len(gen_imgs)):
        # single_img = gen_imgs[im].unsqueeze(im)   # shape (1, C, H, W)
        single_img = gen_imgs[im]   # shape (1, C, H, W)
        save_path = f"images/epoch_{epoch}_{im}.png"
        save_image(single_img, save_path, normalize=True)

[Epoch 0/1000] [Batch 0/1] [D loss: 0.6945] [G loss: 0.6926] [D score: 0.4990] [G score: 0.5003] [PSNR: 11.2429] [SSIM: 0.0320]
[Epoch 1/1000] [Batch 0/1] [D loss: 0.6945] [G loss: 0.6926] [D score: 0.4990] [G score: 0.5003] [PSNR: 10.4647] [SSIM: 0.0265]
[Epoch 2/1000] [Batch 0/1] [D loss: 0.6945] [G loss: 0.6926] [D score: 0.4990] [G score: 0.5003] [PSNR: 10.5514] [SSIM: 0.0322]
[Epoch 3/1000] [Batch 0/1] [D loss: 0.6945] [G loss: 0.6926] [D score: 0.4990] [G score: 0.5003] [PSNR: 9.8459] [SSIM: 0.0178]
[Epoch 4/1000] [Batch 0/1] [D loss: 0.6945] [G loss: 0.6927] [D score: 0.4990] [G score: 0.5002] [PSNR: 9.5588] [SSIM: 0.0238]
[Epoch 5/1000] [Batch 0/1] [D loss: 0.6945] [G loss: 0.6926] [D score: 0.4990] [G score: 0.5003] [PSNR: 10.8684] [SSIM: 0.0317]
[Epoch 6/1000] [Batch 0/1] [D loss: 0.6945] [G loss: 0.6926] [D score: 0.4990] [G score: 0.5003] [PSNR: 11.5767] [SSIM: 0.0329]
[Epoch 7/1000] [Batch 0/1] [D loss: 0.6945] [G loss: 0.6925] [D score: 0.4990] [G score: 0.5003] [PSNR: 11

KeyboardInterrupt: 